# SSS variability

In [1]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

In [2]:
runsdir = '/archive/Raphael.Dussin/runs_forcings'

In [3]:
gpcp = xr.open_zarr(f'{runsdir}/OM4_05.GPCP/bug_rainsnow/ocean_daily', consolidated=True).chunk({'time': 365})

In [4]:
cmt = xr.open_zarr(f'{runsdir}/OM4_05.BlendPrecip/ocean_daily', consolidated=True).chunk({'time': 365})

In [5]:
erai = xr.open_zarr(f'{runsdir}/OM4_05.ERAinterim/ocean_daily', consolidated=True).chunk({'time': 365})

In [6]:
gpcp_0netfw = xr.open_zarr(f'{runsdir}/OM4_05.GPCP_0netfw/bug_rainsnow/ocean_daily', consolidated=True).chunk({'time': 365})

In [7]:
cmt_0netfw = xr.open_zarr(f'{runsdir}/OM4_05.Blend_precip_0netfw/ocean_daily', consolidated=True).chunk({'time': 365})

In [8]:
erai_0netfw = xr.open_zarr(f'{runsdir}/OM4_05.ERAinterim_0netfw/ocean_daily', consolidated=True).chunk({'time': 365})

In [9]:
grid = xr.open_zarr(f'{runsdir}/OM4_05.ERAinterim_0netfw/ocean_static', consolidated=True)

In [ ]:
def rmse(ds, var="sos"):
    "compute anomaly to each year and RMS"
    yearly_mean = ds[var].groupby(ds.time.dt.year).mean(dim="time")
    anom_to_yearly = ds[var].groupby(ds.time.dt.year) - yearly_mean
    mse = (anom_to_yearly*anom_to_yearly).mean(dim='time')
    rmse = xr.apply_ufunc(np.sqrt, mse, dask='parallelized',output_dtypes=mse.dtype)
    return rmse

In [10]:
def intra_monthly(ds, var="sos"):
    "compute anomaly to each month and RMS"
    import pandas as pd
    year_month_idx = pd.MultiIndex.from_arrays([ds.time.dt.year.values, ds.time.dt.month.values])
    ds.coords['year_month'] = ('time', year_month_idx)
    std_allmonth = ds[var].groupby('year_month').std(dim='time')
    avg_std = std_allmonth.mean(dim='year_month')
    return avg_std

In [11]:
std_cmt = intra_monthly(cmt, var="sos")
std_cmt_0net = intra_monthly(cmt_0netfw, var="sos")

In [12]:
std_gpcp = intra_monthly(gpcp, var="sos")
std_gpcp_0net = intra_monthly(gpcp_0netfw, var="sos")

In [13]:
std_erai = intra_monthly(erai, var="sos")
std_erai_0net = intra_monthly(erai_0netfw, var="sos")

## Dask Cluster

In [14]:
from dask.distributed import Client

client = Client("tcp://140.208.147.156:42045")
client

Client Scheduler: tcp://140.208.147.156:42045 Dashboard: http://140.208.147.156:28787/status,Cluster Workers: 16 Cores: 16 Memory: 96.00 GB


In [15]:
_ = std_cmt.load()
_ = std_gpcp.load()
_ = std_erai.load()

_ = std_cmt_0net.load()
_ = std_gpcp_0net.load()
_ = std_erai_0net.load()

KilledWorker: ("('xarray-sos-f357325a65e96b6156fff42b93a5d660', 684, 0, 0)", <Worker 'tcp://140.208.147.30:36569', name: 5-4, memory: 0, processing: 32>)

In [ ]:
#std_cmt.plot(figsize=[12,10], vmin=0, vmax=0.15)

In [ ]:
#std_gpcp.plot(figsize=[12,10], vmin=0, vmax=0.15)

In [ ]:
fig, axs = plt.subplots(ncols=2, nrows=3, figsize=(10, 9),
                        subplot_kw=dict(projection=ccrs.Robinson(central_longitude=205)))
gs = axs[1, 1].get_gridspec()

# remove the underlying axes
for ax in axs[0, 0:]:
    ax.remove()
#axbig = fig.add_subplot(gs[0, 0:], projection=ccrs.Robinson(central_longitude=205))
axbig = fig.add_subplot(gs[0, 0:])

#axbig.pcolormesh(WOA_SSS.lon, WOA_SSS.lat, WOA_SSS.squeeze(),
#                 cmap=cmocean.cm.haline, vmin=30, vmax=40, transform=ccrs.PlateCarree())

C1 = axs[1,0].pcolormesh(grid.geolon_c, grid.geolat_c, (std_erai - std_gpcp).values,
                    cmap='bwr', vmin=-0.02, vmax=0.02, transform=ccrs.PlateCarree())

cbar = plt.colorbar(C1, orientation='horizontal', ticks=[-0.02,-0.01,0,0.01,0.02])
cbar.ax.xaxis.set_ticks_position('top')
cbar.ax.set_xticklabels(['-0.02', '-0.01', '0', '0.01', '0.02'], fontsize=20)

axs[1,1].pcolormesh(grid.geolon_c, grid.geolat_c, (std_cmt - std_gpcp).values,
                    cmap='bwr', vmin=-0.02, vmax=0.02, transform=ccrs.PlateCarree())

axs[2,0].pcolormesh(grid.geolon_c, grid.geolat_c, (std_erai - std_erai_0net).values,
                    cmap='bwr', vmin=-0.02, vmax=0.02, transform=ccrs.PlateCarree())

axs[2,1].pcolormesh(grid.geolon_c, grid.geolat_c, (std_cmt - std_cmt_0net).values,
                    cmap='bwr', vmin=-0.02, vmax=0.02, transform=ccrs.PlateCarree())



#axs[3,0].pcolormesh(SSS_bias_GPCP.lon, SSS_bias_GPCP.lat, SSS_bias_GPCP.squeeze(),
#                    cmap='bwr', vmin=-2, vmax=2, transform=ccrs.PlateCarree())
#axs[3,1].pcolormesh(SSS_bias_GPCP0.lon, SSS_bias_GPCP0.lat, SSS_bias_GPCP0.squeeze(),
#                    cmap='bwr', vmin=-2, vmax=2, transform=ccrs.PlateCarree())


# hide the big plot so we only see the colorbar
axbig.set_visible(False)

axs[1,0].coastlines()
axs[1,1].coastlines()
axs[2,0].coastlines()
axs[2,1].coastlines()
#axs[3,0].coastlines()
#axs[3,1].coastlines()

axs[1,0].set_facecolor([0.5,0.5,0.5])
axs[1,1].set_facecolor([0.5,0.5,0.5])
axs[2,0].set_facecolor([0.5,0.5,0.5])
axs[2,1].set_facecolor([0.5,0.5,0.5])
#axs[3,0].set_facecolor([0.5,0.5,0.5])
#axs[3,1].set_facecolor([0.5,0.5,0.5])

axs[1,0].set_title('ERAi - GPCP', fontsize=20, weight='bold')
axs[1,1].set_title('CMT - GPCP', fontsize=20, weight='bold')
axs[2,0].set_title('ERAi - ERAi0FW', fontsize=20, weight='bold')
axs[2,1].set_title('CMT - CMT0FW', fontsize=20, weight='bold')

axs[1,0].text(15, -65, '$(a)$',
            verticalalignment='top',
            horizontalalignment='left',
            transform=ccrs.PlateCarree(),
            bbox={'facecolor': 'black'},
            color='white', fontsize=20)

axs[1,1].text(15, -65, '$(b)$',
            verticalalignment='top',
            horizontalalignment='left',
            transform=ccrs.PlateCarree(),
            bbox={'facecolor': 'black'},
            color='white', fontsize=20)

axs[2,0].text(15, -65, '$(c)$',
            verticalalignment='top',
            horizontalalignment='left',
            transform=ccrs.PlateCarree(),
            bbox={'facecolor': 'black'},
            color='white', fontsize=20)

axs[2,1].text(15, -65, '$(d)$',
            verticalalignment='top',
            horizontalalignment='left',
            transform=ccrs.PlateCarree(),
            bbox={'facecolor': 'black'},
            color='white', fontsize=20)

plt.savefig('SSS_variance_intra_month_multi.png', bbox_inches='tight', facecolor='w', edgecolor='w', transparent=False)

In [123]:
#(std_cmt - std_gpcp).plot(figsize=[16,10], vmin=-0.02, vmax=0.02, cmap='bwr')

In [124]:
#(std_cmt - std_cmt_0net).plot(figsize=[16,10], vmin=-0.02, vmax=0.02, cmap='bwr')

In [125]:
#(std_erai - std_erai_0net).plot(figsize=[16,10], vmin=-0.02, vmax=0.02, cmap='bwr')

In [126]:
#(std_gpcp - std_gpcp_0net).plot(figsize=[16,10], vmin=-0.02, vmax=0.02, cmap='bwr')

In [127]:
#(std_erai - std_cmt).plot(figsize=[16,10], vmin=-0.02, vmax=0.02, cmap='bwr')

In [128]:
#(std_erai - std_gpcp).plot(figsize=[16,10], vmin=-0.02, vmax=0.02, cmap='bwr')

In [129]:
#(std_erai_0net - std_gpcp_0net).plot(figsize=[16,10], vmin=-0.02, vmax=0.02, cmap='bwr')